In [ ]:
import sys, os
print(f"🐍 Python Interpreter: {sys.executable}")
print(f"📂 Working Directory: {os.getcwd()}")
try:
    import numpy as np
    import biotite
    print(f"✅ Dependencies found: numpy {np.__version__}, biotite {biotite.__version__}")
except ImportError as e:
    print(f"❌ Missing Dependency: {e}")
    print("HINT: Ensure you are running in the correct virtual environment.")


# 🎓 The Virtual NMR Spectrometer

Welcome to the **Interactive Relaxation Lab**! 

This notebook simulates a **Nuclear Magnetic Resonance (NMR)** experiment on a virtual protein.
NMR relaxation rates ($R_1$, $R_2$, and Heteronuclear NOE) are the primary tools biophysicists use to study protein movement at the atomic level.

### 🎯 Goal
Understand how **molecular size** (tumbling rate) and **magnetic field strength** affect the signals we measure. This simulation uses the standard *Lipari-Szabo Model-Free* formalism, which is the gold standard for analyzing protein dynamics.

### ⚠️ How to Run (Important!)
This notebook uses advanced physics libraries that require a specific environment setup. Follow these steps strictly:

1.  **Run All Cells** (`Runtime` -> `Run all` or `Ctrl+F9`).
2.  **Wait for the Crash**: The first cell (`SETUP`) will install libraries and then **automatically crash/restart** the session. This is normal and required to load the correct components.
3.  **Wait 10 Seconds**: Allow the session to reconnect.
4.  **Run All Cells AGAIN**: This time, the setup will detect it is ready ('✅ Dependencies Ready') and proceed typically.

> **Note**: The step labeled *'Minimizing energy'* runs a full molecular dynamics simulation. On Google Colab (CPU), **this may take 8+ minutes**. Please be patient while it builds your custom protein structure!


In [ ]:
# 🛠️ SETUP: Install dependencies (Auto-Restart Mode)
# This cell automates the 'Factory Reset' to guarantee NumPy stability.

import os
import sys

# File marker to prevent infinite loops
if not os.path.exists("installed.marker"):
    try:
        import google.colab
        IN_COLAB = True
    except ImportError:
        IN_COLAB = False

    if IN_COLAB:
        print("🛡️ Locking Environment to NumPy 1.x...")
        
        # 1. Create constraint file (We effectively bully pip into submission here)
        with open("constraints.txt", "w") as f:
            f.write('numpy<2.0')
        
        # 2. Install with explicit constraints
        # We ignore 'shap' errors because we don't use it.
        !pip install -q "numpy<2.0" -c constraints.txt
        !pip install -q biotite -c constraints.txt
        !pip install -q openmm matplotlib ipywidgets py3Dmol -c constraints.txt
        !pip install -q --no-deps git+https://github.com/elkins/synth-pdb.git@master
        
        # 3. Create marker
        with open("installed.marker", "w") as f:
            f.write("done")
        
        print("🔄 Installation complete. KERNEL RESTARTING AUTOMATICALLY...")
        print("⚠️ Please wait 10 seconds, then Run All Cells again.")
        
        # 4. Kill the kernel to force reload of new C-extensions
        os.kill(os.getpid(), 9)
else:
    import numpy
    print(f"✅ Dependencies Ready. NumPy: {numpy.__version__}")


In [ ]:
# This cell ensures the code handles missing OpenMM gracefully.
import os
import sys
import synth_pdb
import base64
import importlib

try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    # Let's try without the hotfix, since the required features have migrated to PyPi
    print("✅ Installing synth-pdb from PyPi with pip.")
    !pip install synth-pdb
else:
    print("✅ Running local synth-pdb so no installation required.")

In [ ]:
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
import numpy as np
from synth_pdb.generator import generate_pdb_content
from synth_pdb.relaxation import calculate_relaxation_rates, predict_order_parameters
import biotite.structure.io.pdb as pdb
import io

# 1. Generate a Test Protein (Zinc Finger-like Motif)
# Visualization: Beta Hairpin (Sheet-Turn-Sheet) + Alpha Helix
# This provides a rich mix of rigid and flexible regions.
sequence = "VKITVGGTLTVALGGALALALALALALAA"
structure_def = "1-5:beta,6-8:random,9-13:beta,14-16:random,17-29:alpha"

print("🧬 Generating virtual protein (Diet Zinc Finger - now with 0% actual Zinc!)...")
print("   - Optimizing side-chains (Monte Carlo)...")
print("   - Minimizing energy (OpenMM if available)...")

pdb_content = generate_pdb_content(
    sequence_str=sequence, 
    structure=structure_def, 
    optimize_sidechains=True, 
    minimize_energy=True
)
pdb_file = pdb.PDBFile.read(io.StringIO(pdb_content))
structure = pdb_file.get_structure(model=1)

# Pre-calculate Order Parameters (S2) based on structure
s2_map = predict_order_parameters(structure)
res_ids = sorted(list(s2_map.keys()))
s2_values = [s2_map[r] for r in res_ids]

print("✅ Protein Model Ready!")


### 🧬 The Simulated Protein (Zinc Finger Motif)

We have generated a synthetic **Zinc Finger-like** fold to demonstrate contrast between different secondary structures:
*   **Residues 1-5**: Beta Strand 1 (Rigid, Extended)
*   **Residues 6-8**: Turn (Flexible)
*   **Residues 9-13**: Beta Strand 2 (Rigid, Extended)
*   **Residues 14-16**: Loop (Flexible)
*   **Residues 17-29**: Alpha Helix (Rigid, Compact)

Notice how the physics engine (`synth-pdb`) automatically assigns different **Order Parameters ($S^2$)** to these regions. $S^2$ represents spatial restriction: **1.0** is completely rigid, **0.0** is completely disordered.

> ⚠️ **Troubleshooting: Weird Shapes?**
> 
> Occasionally, the energy minimization process generates a 'post-modern art masterpiece' instead of a protein. If your molecule looks like exploded spaghetti, you can generate a new one.
> 
> **If this happens**: Please **Restart Runtime** and **Run All** to try a fresh simulation.


In [ ]:
# Visualizing the Structure
import py3Dmol
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import matplotlib
import numpy as np

def pdb_to_bfactor_map(pdb_str):
    """Extract average B-factor per residue."""
    res_b = {}
    counts = {}
    for line in pdb_str.splitlines():
        if line.startswith("ATOM"):
            try:
                res_id = int(line[22:26].strip())
                b = float(line[60:66].strip())
                if res_id not in res_b:
                    res_b[res_id] = 0.0
                    counts[res_id] = 0
                res_b[res_id] += b
                counts[res_id] += 1
            except ValueError:
                pass
    # Average
    for r in res_b:
        if counts[r] > 0:
            res_b[r] /= counts[r]
    return res_b

res_b_map = pdb_to_bfactor_map(pdb_content)
if not res_b_map:
    res_b_map = {i: 50 for i in range(1, 30)}

vals = list(res_b_map.values())
min_val = min(vals)
max_val = max(vals)

print(f"📏 Data Stats: Min={min_val:.1f}, Max={max_val:.1f}, Mean={np.mean(vals):.1f}")

# Force Physics Range Scaling
# Prevents one outlier (e.g. 100) from squashing all 20s to Blue.
vmin = 15.0  # Rigid Limit
vmax = 65.0  # Flexible Limit

# Setup Viewer
view = py3Dmol.view(width=400, height=300)
view.addModel(pdb_content, 'pdb')

# Apply Logic: Python-calculated Colors
try:
    cmap = matplotlib.colormaps['bwr']
except AttributeError:
    cmap = plt.get_cmap('bwr')

# Clip values to range for color lookup
norm = mcolors.Normalize(vmin=vmin, vmax=vmax)

view.setStyle({'cartoon': {'color': 'white'}}) # Base

for r, b_val in res_b_map.items():
    rgba = cmap(norm(b_val))
    hex_color = mcolors.to_hex(rgba)
    view.addStyle({'resi': r}, {'cartoon': {'color': hex_color}})

view.zoomTo()
print("\n--- COLOR GUIDE ---")
print("🔵 BLUE (Cool) = RIGID (Frozen in existential dread - Helices/Sheets)")
print("🔴 RED  (Hot)  = FLEXIBLE (Wacky Waving Inflatable Tube Guy - Loops)")
print("-------------------")
view.show()


### 📊 Guide to the Plots

When you run the simulator below, you will see three coupled plots. Here is how to interpret them:

1.  **$R_1$ (Longitudinal Rate - Blue)**: 
    *   *Physics*: Sensitive to fast interaction fluctuations (nanosecond scale).
    *   *Pattern*: Often relatively flat for folded proteins, but may dip in flexible loops.

2.  **$R_2$ (Transverse Rate - Red)**: 
    *   *Physics*: Sensitive to slow motions (global tumbling) and chemical exchange.
    *   *Key Insight*: **$R_2$ scales with protein size**. Large proteins (slow tumbling) have high $R_2$ rates. High $R_2$ means the signal decays fast (broad lines), making large proteins hard to study!
    *   *Flexible Regions*: $R_2$ drops sharply because local flexibility averages out the magnetic interactions.

3.  **Heteronuclear NOE (Green)**: 
    *   *The Rigidity Sensor*: This is the most robust indicator of local structure.
    *   **Value ~ 0.8**: Rigid backbone (Helix/Sheet).
    *   **Value < 0.6**: Flexible loop/terminus.
    *   **Negative Values**: Extremely flexible or unfolded.

---


In [ ]:
# Interactive Relaxation Dynamics Exploration
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import io

# 1. Create the persistent Image widget
# We will update this widget's value, rather than printing to the log.
plot_image = widgets.Image(format='png', width=800, height=1000)

# 2. Create Sliders
field_slider = widgets.IntSlider(min=400, max=1200, step=100, value=600, description='Field (MHz)')
tumbling_slider = widgets.FloatSlider(min=2.0, max=50.0, step=1.0, value=10.0, description='Tumbling (ns)')

# 3. Update Function
def update_relaxation_plot(change=None):
    field_mhz = field_slider.value
    tau_m_ns = tumbling_slider.value
    
    # Stop Matplotlib from trying to be helpful
    plt.ioff()
    
    # Calculate
    rates = calculate_relaxation_rates(
        structure, 
        field_mhz=field_mhz, 
        tau_m_ns=tau_m_ns, 
        s2_map=s2_map
    )
    r1 = [rates[r]['R1'] for r in res_ids]
    r2 = [rates[r]['R2'] for r in res_ids]
    noe = [rates[r]['NOE'] for r in res_ids]
    
    # Plot to Memory
    # Note: We create a new figure each time but NEVER display it directly.
    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(10, 12), sharex=True)
    
    ax1.plot(res_ids, r1, 'o-', color='blue', label='R1 (Longitudinal)')
    ax1.set_ylabel('$R_1$ ($s^{-1}$)')
    ax1.set_title(f'Relaxation Dynamics @ {field_mhz} MHz, $\\tau_m$={tau_m_ns}ns')
    ax1.grid(True, alpha=0.3)
    
    ax2.plot(res_ids, r2, 's-', color='red', label='R2 (Transverse)')
    ax2.set_ylabel('$R_2$ ($s^{-1}$)')
    ax2.grid(True, alpha=0.3)
    
    ax3.plot(res_ids, noe, '^-', color='green', label='HetNOE')
    ax3.set_ylabel('NOE Ratio')
    ax3.set_xlabel('Residue Number')
    ax3.grid(True, alpha=0.3)
    
    for r in res_ids:
        if s2_map[r] < 0.75:
            for ax in [ax1, ax2, ax3]:
                ax.axvspan(r-0.5, r+0.5, color='yellow', alpha=0.1)
    
    plt.tight_layout()
    
    # Save to buffer
    buf = io.BytesIO()
    fig.savefig(buf, format='png')
    plt.close(fig) # Essential: kill the object
    
    # Update Image Widget
    buf.seek(0)
    plot_image.value = buf.read()

# 4. Bind Events
# continuous_update=False prevents lag/stacking from rapid drags
field_slider.continuous_update = False
tumbling_slider.continuous_update = False

field_slider.observe(update_relaxation_plot, names='value')
tumbling_slider.observe(update_relaxation_plot, names='value')

# 5. Initial Render
update_relaxation_plot()

# 6. Layout UI
ui = widgets.VBox([field_slider, tumbling_slider])
display(ui, plot_image)


### 🧪 Try These Experiments

Use the sliders above to change the "experimental" conditions:

1.  **Simulate a Giant Protein Complex**:
    *   Slide **Tumbling (ns)** to **40.0 ns**.
    *   *Observation*: Look at the **$R_2$ (Red)** plot. It shoots up significantly! This is why NMR is limited to smaller proteins (< 50-100 kDa) without special tricks (like TROSY).
    *   *Note*: The NOE profile stays mostly the same—local rigidity hasn't changed, only the global tumbling.

2.  **Go to High Field**:
    *   Slide **Field (MHz)** from **600** to **1200**.
    *   *Observation*: $R_2$ increases. This is due to **Chemical Shift Anisotropy (CSA)** becoming a stronger relaxation mechanism at high magnetic fields.


### ⚛️ Primer: What is "Chemical Shift"?

Before jumping into the spectra, let's understand the core concept: **Chemical Shift** ($\delta$).

Imagine every atom in the protein is a tiny radio transmitter. If they were all in a vacuum, they would all broadcast at the exact same frequency (determined by the big magnet).

**But they are not alone.** They are surrounded by electrons.

*   **The Shielding Effect**: Electrons orbit the nucleus and create their own tiny magnetic fields that oppose the big magnet. This "shields" the nucleus.
*   **The Environment Matters**: 
    *   An atom in a **Beta Sheet** has a different electron cloud shape than one in an **Alpha Helix**.
    *   This means they are "shielded" differently.
    *   Therefore, they "feel" a slightly different net magnetic field and broadcast at a **different frequency**.

This difference is the **Chemical Shift**. We measure it in **ppm** (parts per million) so that the numbers stay the same regardless of whether you use a huge 1200 MHz magnet or a smaller 600 MHz one.

---


# 🔬 Advanced: The Protein Fingerprint (HSQC)

While Relaxation tells us about **Motion**, the **HSQC** (Heteronuclear Single Quantum Coherence) spectrum tells us about **Identity** and **Fold**.

### 🙋‍♂️ The "Roll Call" Analogy
Imagine an HSQC experiment as a **roll call**. Every amino acid in the protein has one N-H bond in its backbone. In a strong magnetic field, this N-H bond acts like a radio antenna.

In an HSQC spectrum, **every dot is one amino acid raising its hand**.

*   **Note**: **Proline is the introvert**. It refuses to participate in the HSQC roll call (no amide proton). It's there, silently judging the others.

### 🌈 Reading the Map
*   **Spread Out Dots**: The protein is **FOLDED**. The chemical environment around each residue is unique (some are buried next to aromatics, some are exposed to water), so they all shout at slightly different frequencies. It's like a diverse choir.
*   **Clumped Dots**: The protein is **UNFOLDED**. Every residue sees the same boring water environment. They all pile on top of each other in the center of the plot. It's like everyone wearing the same beige uniform.
*   **Color (Shift Index)**: 
    *   **Blue/Upfield**: Alpha Helix signals often shift 'right' and 'up'.
    *   **Red/Downfield**: Beta Sheet signals often shift 'left' and 'down'.


In [ ]:
# Calculate J-Couplings using the Karplus Equation
from synth_pdb.j_coupling import calculate_hn_ha_coupling

# 1. Calculate
j_couplings = calculate_hn_ha_coupling(structure)

# 2. Extract Data
res_nums = []
j_vals = []
colors = []

for r in res_ids:
    if r in j_couplings.get('A', {}):
        val = j_couplings['A'][r]
        res_nums.append(r)
        j_vals.append(val)
        # Color by physics expectation
        if val < 6.0: colors.append('blue') # Helix-like
        elif val > 8.0: colors.append('red') # Sheet-like
        else: colors.append('gray') # Random/Avg

# 3. Plot
plt.figure(figsize=(10, 4))
plt.bar(res_nums, j_vals, color=colors, alpha=0.7, edgecolor='black')
plt.axhline(4.0, color='blue', linestyle='--', alpha=0.5, label='Alpha Helix (~4 Hz)')
plt.axhline(9.0, color='red', linestyle='--', alpha=0.5, label='Beta Sheet (~9 Hz)')
plt.xlabel('Residue Number')
plt.ylabel('$^3J_{HNH\\alpha}$ (Hz)')
plt.title('Backbone Dihedral Probe: Scalar Couplings')
plt.legend()
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

print("📏 INTERPRETATION:")
print("Smaller couplings (< 6 Hz) indicate Alpha-Helical turns.")
print("Larger couplings (> 8 Hz) indicate Extended/Beta conformations.")


In [ ]:
# Simulate 2D HSQC Spectrum
from synth_pdb.chemical_shifts import predict_chemical_shifts

# 1. Predict Shifts (SPARTA-lite)
shifts = predict_chemical_shifts(structure)

# 2. Collect (H, N) pairs
h_shifts = []
n_shifts = []
labels = []
ss_colors = []

from synth_pdb.structure_utils import get_secondary_structure
ss_types = get_secondary_structure(structure)

for i, r in enumerate(res_ids):
    chain_shifts = shifts.get('A', {})
    res_shifts = chain_shifts.get(r, {})
    
    if 'H' in res_shifts and 'N' in res_shifts:
        h = res_shifts['H']
        n = res_shifts['N']
        h_shifts.append(h)
        n_shifts.append(n)
        labels.append(f"{structure[structure.res_id==r].res_name[0]}{r}")
        
        # Color by SS
        ss = ss_types[i] if i < len(ss_types) else 'coil'
        if ss == 'alpha': ss_colors.append('blue')
        elif ss == 'beta': ss_colors.append('red')
        else: ss_colors.append('gray')

# 3. Plot HSQC
plt.figure(figsize=(8, 8))
plt.scatter(h_shifts, n_shifts, c=ss_colors, s=100, alpha=0.8, edgecolors='black')

# Standard NMR axes orientation (High -> Low)
plt.xlim(10.5, 6.5)  # Proton: 10 down to 6
plt.ylim(135, 100)   # Nitrogen: 135 down to 100

plt.xlabel('$^{1}$H Chemical Shift (ppm)')
plt.ylabel('$^{15}$N Chemical Shift (ppm)')
plt.title('Synthetic 2D $^{1}$H-$^{15}$N HSQC Spectrum')

# Label peaks
for h, n, txt in zip(h_shifts, n_shifts, labels):
    plt.annotate(txt, (h, n), xytext=(3, 3), textcoords='offset points', fontsize=8)

plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("📊 INTERPRETATION:")
print("Each dot represents one residue's backbone amide group.")
print("Dispersion (spread) indicates a folded structure.")
print("Collapsed peaks in the center would indicate an unfolded/disordered protein.")


### 🧲 Primer: What is "NOESY"?

**NOESY** stands for **Nuclear Overhauser Effect Spectroscopy**.

Think of it as **"Molecular Stalking"**.

*   **Through-Space vs. Through-Bond**: Most NMR experiments rely on bonds. The **NOE** is different—it measures how much atoms are **whispering to each other** through space (usually < 5 Angstroms).
*   **The Ruler of Structure**: The strength of the NOE signal is proportional to $\frac{1}{r^6}$, where $r$ is the distance between atoms. This means it is incredibly sensitive to distance.
*   **Folding**: If we see an NOE signal between Residue 1 and Residue 100, we know the protein chain must loop back around so those two residues are touching! This is the primary data used to calculate 3D structures.

---


### 🗺️ 3. The Contact Map (Virtual NOESY)

The **Contact Map** is the "Fingerprint" of a protein fold. It shows which residues are touching in 3D space.

*   **Diagonal**: Residues are always close to their neighbors (i, i+1).
*   **Off-Diagonal**: These are the interesting **Long-Range Contacts**. They tell us the protein has folded back on itself (e.g., a Beta Sheet or Alpha Helix packing).
*   **AI Relevance**: This 2D matrix is exactly what tools like **AlphaFold** predict internally (the "Distogram").


In [ ]:
# Calculate Contact Map
from synth_pdb.contact import compute_contact_map

# Compute Alpha-Carbon distances (Standard for Fold Recognition)
contact_map = compute_contact_map(structure, method='ca', power=None)
binary_map = compute_contact_map(structure, method='ca', threshold=8.0, power=0)

# Plotting
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# 1. Distance Matrix (Continuous)
im1 = ax1.imshow(contact_map, cmap='viridis_r', interpolation='nearest')
ax1.set_title('Distance Matrix (Angstroms)')
ax1.set_xlabel('Residue Index')
ax1.set_ylabel('Residue Index')
plt.colorbar(im1, ax=ax1, label='Distance (Å)')

# 2. Binary Contact Map (AI/NMR Style)
# Threshold < 8 Angstroms is standard for CASP/AlphaFold evaluation
im2 = ax2.imshow(binary_map, cmap='Greys', interpolation='nearest')
ax2.set_title('Binary Contact Map (Threshold < 8Å)')
ax2.set_xlabel('Residue Index')
ax2.set_yticks([]) # Hide Y ticks for cleanliness

plt.tight_layout()
plt.show()

print("📊 INTERPRETATION:")
print("Dark patterns away from the diagonal indicate FOLDED structure.")
print("A Thick Diagonal with no off-diagonal spots = DISORDERED/UNFOLDED chain.")


### 📊 3. Chemical Shift Index (CSI)

The **Chemical Shift Index** is a method to visualize secondary structure directly from the raw data, without calculating a full 3D structure.

We plot the deviation of the Alpha-Carbon ($C_\alpha$) shift from its random coil value:
$$ \Delta \delta = \delta_{measured} - \delta_{random\_coil} $$

*   **Positive Clusters (> +0.7 ppm)**: Indicate **Alpha Helices**.
*   **Negative Clusters (< -0.7 ppm)**: Indicate **Beta Sheets**.
*   **Near Zero**: Indicates **Random Coil / Loops**.


In [ ]:
# Calculate CSI
import importlib
import synth_pdb.chemical_shifts
# Force reload to pick up the new function definition from disk
importlib.reload(synth_pdb.chemical_shifts)
from synth_pdb.chemical_shifts import calculate_csi

# 1. Get Deviations
csi_data = calculate_csi(shifts, structure)
chain_id = list(csi_data.keys())[0]
deltas = csi_data[chain_id]

res_nums = sorted(deltas.keys())
values = [deltas[r] for r in res_nums]

# 2. Plotting
plt.figure(figsize=(10, 4))
colors = ['red' if v > 0 else 'blue' for v in values]
plt.bar(res_nums, values, color=colors, alpha=0.7)

# 3. Add Threshold Lines (The "CSI" definition)
plt.axhline(0.7, color='black', linestyle='--', linewidth=1, label='Helix Threshold (+0.7)')
plt.axhline(-0.7, color='black', linestyle=':', linewidth=1, label='Sheet Threshold (-0.7)')
plt.axhline(0, color='black', linewidth=0.5)

plt.title(f'Chemical Shift Index (CSI) - $C_\\alpha$ Deviations')
plt.ylabel('$\\Delta \\delta$ (ppm)')
plt.xlabel('Residue Number')
plt.legend()
plt.grid(axis='y', alpha=0.3)
plt.show()


### 🛡️ 4. Structure Quality & Validation

A structure is only useful if it obeys physics. We run a **Full Validation Suite** covering:
1.  **Bond Geometries**: Lengths and angles.
2.  **Ramachandran Plot**: Ensuring backbone torsion angles ($\phi, \psi$) are in allowed regions (alpha/beta/proline-restricted).
3.  **Steric Clashes**: Atoms overlapping in space.
4.  **Rotamers**: Side-chains in low-energy conformations.


In [ ]:
# Run Validation
from synth_pdb.validator import PDBValidator
import logging

# Configure logging to show only critical info in notebook
logging.basicConfig(level=logging.INFO, stream=sys.stdout, format='%(levelname)s: %(message)s', force=True)

print("🔍 Running Biophysical Validation...")
validator = PDBValidator(pdb_content)
validator.validate_all()

violations = validator.get_violations()
if not violations:
    print("✅ No Violations Found! Structure is physically sound.")
else:
    print(f"⚠️ Found {len(violations)} Violations:")
    for v in violations[:10]: # Show first 10
        print(f"  - {v}")
    if len(violations) > 10:
        print(f"  ... and {len(violations)-10} more.")


In [ ]:
# Ramachandran Plot Visualization
# We extract Phi/Psi angles and plot them against the "Allowed" regions.

phi_angles = []
psi_angles = []
res_colors = []

# Extract Dihedrals (using Biotite)
from biotite.structure import dihedral_backbone
phi, psi, omega = dihedral_backbone(structure)

# Convert radians to degrees and filter non-existing (termini)
for i in range(len(phi)):
    if not np.isnan(phi[i]) and not np.isnan(psi[i]):
        p = np.degrees(phi[i])
        s = np.degrees(psi[i])
        phi_angles.append(p)
        psi_angles.append(s)
        
        # Color by Secondary Structure
        # Note: 'structure' array index matches phi/psi index
        # We'll use a simple heuristic for color
        if -100 < p < -30 and -80 < s < -10: 
            res_colors.append('blue') # Alpha
        elif -180 < p < -40 and (90 < s < 180 or -180 < s < -160):
             res_colors.append('red') # Beta
        elif p > 0: 
             res_colors.append('green') # Left-handed (Gly check)
        else:
             res_colors.append('gray')

plt.figure(figsize=(6, 6))
plt.scatter(phi_angles, psi_angles, c=res_colors, alpha=0.7, edgecolors='black')

# Underlying Regions (Simplified Boxes for reference)
plt.gca().add_patch(plt.Rectangle((-100, -70), 70, 60, color='blue', alpha=0.1, label='Alpha'))
plt.gca().add_patch(plt.Rectangle((-180, 90), 140, 90, color='red', alpha=0.1, label='Beta'))

plt.xlim(-180, 180)
plt.ylim(-180, 180)
plt.axhline(0, color='black', linewidth=0.5)
plt.axvline(0, color='black', linewidth=0.5)
plt.xlabel(r'Phi ($\phi$)')
plt.ylabel(r'Psi ($\psi$)')
plt.title('Ramachandran Plot (Backbone Geometry)')
plt.grid(True, alpha=0.3)
plt.legend(loc='upper right')
plt.show()

print("📊 INTERPRETATION:")
print("Blue Dots: Alpha Helices (Bottom Left)")
print("Red Dots: Beta Sheets (Top Left)")
print("Green Dots: Left-handed Helices (Top Right - Usually only Glycine)")
